In [1]:
pip install kaggle

In [2]:
# import libarys
import sqlite3
import os
import zipfile
import pandas as pd

In [3]:
# download the dataset from Kaggle
!kaggle datasets download -d tmdb/tmdb-movie-metadata

Dataset URL: https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata
License(s): other
tmdb-movie-metadata.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
# extract the dataset
with zipfile.ZipFile("tmdb-movie-metadata.zip", 'r') as zip_ref:
    zip_ref.extractall("tmdb_data")

In [5]:
# load the dataset into a DataFrame
df = pd.read_csv("tmdb_data/tmdb_5000_movies.csv")

In [6]:
# dataset columns in this dataset
print("Columns in the dataset:")
print(df.columns)

Columns in the dataset:
Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')


In [7]:
# create SQLite database
conn = sqlite3.connect('movies.db')

In [8]:
# load data into database
df.to_sql('movies', conn, if_exists='replace', index=False)

4803

In [9]:
# top 5 Movies by revenue
print("\nQuery 1: Find the top 5 movies by revenue")
query_1 = """
SELECT title, revenue
FROM movies
ORDER BY revenue DESC
LIMIT 5
"""

result_1 = pd.read_sql_query(query_1, conn)
print(result_1)


Query 1: Find the top 5 movies by revenue
            title     revenue
0          Avatar  2787965087
1         Titanic  1845034188
2    The Avengers  1519557910
3  Jurassic World  1513528810
4       Furious 7  1506249360


In [10]:
# analyze by average revenue per genre
print("\nQuery 2: Calculate the average revenue per genre")
query_2 = """
SELECT genres, AVG(revenue) AS avg_revenue
FROM movies
GROUP BY genres
ORDER BY avg_revenue DESC
"""
result_2 = pd.read_sql_query(query_2, conn)
print(result_2)


Query 2: Calculate the average revenue per genre
                                                 genres   avg_revenue
0     [{"id": 10751, "name": "Family"}, {"id": 14, "...  1.025491e+09
1     [{"id": 18, "name": "Drama"}, {"id": 28, "name...  1.004558e+09
2     [{"id": 10751, "name": "Family"}, {"id": 12, "...  9.665506e+08
3     [{"id": 12, "name": "Adventure"}, {"id": 14, "...  9.382127e+08
4     [{"id": 16, "name": "Animation"}, {"id": 35, "...  8.772448e+08
...                                                 ...           ...
1170  [{"id": 10749, "name": "Romance"}, {"id": 18, ...  0.000000e+00
1171  [{"id": 10749, "name": "Romance"}, {"id": 12, ...  0.000000e+00
1172  [{"id": 10402, "name": "Music"}, {"id": 35, "n...  0.000000e+00
1173  [{"id": 10402, "name": "Music"}, {"id": 35, "n...  0.000000e+00
1174  [{"id": 10402, "name": "Music"}, {"id": 27, "n...  0.000000e+00

[1175 rows x 2 columns]


In [11]:
# year with the most successful movies, based on revenue
print("\nQuery 3: Identify the year with the most successful movies (by revenue)")
query_3 = """
SELECT release_date, SUM(revenue) AS total_revenue
FROM movies
GROUP BY SUBSTR(release_date, 1, 4) -- Extract year from release_date
ORDER BY total_revenue DESC
LIMIT 1
"""
result_3 = pd.read_sql_query(query_3, conn)
print(result_3)


Query 3: Identify the year with the most successful movies (by revenue)
  release_date  total_revenue
0   2012-07-16    24141710246


In [12]:
# calculate the average rating per genre
print("\nQuery 5: Calculate the average rating per genre")
query_5 = """
SELECT genres, AVG(vote_average) AS avg_rating
FROM movies
GROUP BY genres
ORDER BY avg_rating DESC
"""
result_5 = pd.read_sql_query(query_5, conn)
print(result_5)


Query 5: Calculate the average rating per genre
                                                 genres  avg_rating
0     [{"id": 37, "name": "Western"}, {"id": 28, "na...         9.3
1     [{"id": 18, "name": "Drama"}, {"id": 28, "name...         8.2
2     [{"id": 14, "name": "Fantasy"}, {"id": 18, "na...         8.2
3     [{"id": 14, "name": "Fantasy"}, {"id": 16, "na...         8.2
4     [{"id": 12, "name": "Adventure"}, {"id": 14, "...         8.2
...                                                 ...         ...
1170  [{"id": 28, "name": "Action"}, {"id": 80, "nam...         2.0
1171  [{"id": 28, "name": "Action"}, {"id": 35, "nam...         1.0
1172  [{"id": 53, "name": "Thriller"}, {"id": 35, "n...         0.0
1173  [{"id": 53, "name": "Thriller"}, {"id": 27, "n...         0.0
1174  [{"id": 28, "name": "Action"}, {"id": 35, "nam...         0.0

[1175 rows x 2 columns]


In [13]:
# correlation calcultaions: budget vs. revenue
print("\nQuery 7: Correlation analysis between budget and revenue")
query_7 = """
SELECT budget, revenue
FROM movies
WHERE budget > 0 AND revenue > 0
"""
result_7 = pd.read_sql_query(query_7, conn)
print("Correlation analysis:")
correlation = result_7.corr() 
print(correlation)


Query 7: Correlation analysis between budget and revenue
Correlation analysis:
           budget   revenue
budget   1.000000  0.705399
revenue  0.705399  1.000000


In [14]:
conn.close()